In [1]:
import numpy as np
import matplotlib.pyplot as plt
from cvxopt import matrix, solvers
from scipy.optimize import minimize
from scipy.linalg import null_space
from scipy.linalg import solve, LinAlgError

In [2]:
# Set the seed for reproducibility
np.random.seed(5)

# Set precision to 4 decimal places for display
# np.set_printoptions(precision=4, suppress=True)

In [3]:
def print_array(arr, decimals=0):
    rows = []
    # Iterate over each row in the array
    for row in arr:
        # Preprocess each element: convert -0.0 to 0.0
        processed_row = [0.0 if x == 0.0 else x for x in row]
        # Format each number with four decimal places, adding a space in front of positive numbers
        formatted_row = " | ".join(f"{' ' if x >= 0 else ''}{x:.{decimals}f}" for x in processed_row)
        rows.append(formatted_row)
    # Join all rows into a single string separated by newlines
    return "\n".join(rows)

def is_positive_definite(A):
    # Check if all eigenvalues are positive
    eigenvalues = np.linalg.eigvals(A)
    return np.all(eigenvalues > 0)

def is_positive_semidefinite(A):
    # Check if all eigenvalues are non-negative
    eigenvalues = np.linalg.eigvals(A)
    return np.all(eigenvalues >= 0)

def spectral_norm(A):
    # Computes the spectral norm of A (largest singular value)
    singular_values = np.linalg.svd(A, compute_uv=False)
    return np.max(singular_values)

def is_y_valid(A, y):
    # Check if euclidean norm of y >= than the spectral norm of some matrix (in our case M)
    return np.linalg.norm(y) >= spectral_norm(A)

def get_initial_W(A, x, b):
    # Constructs the initial working set W_0 which we define to hold all valid constraints of x_0
    W = np.where((A @ x).T >= b)[1].tolist()
    return W
    
def calc_pk(G, c, A, b, x):
    # Calculates p_k by solving the subproblem (16.39) via the Schur-Complement Method from the book
    n = np.shape(x)[0]
    G_inv = np.linalg.inv(G)
    temp_inv = np.linalg.inv((A @ G_inv @ A.T))
    C = G_inv - G_inv @ A.T @ temp_inv @ A @ G_inv
    E = G_inv @ A.T @ temp_inv
    F = -temp_inv
    g = c + G @ x
    h = A @ x - b.reshape(-1, 1)
    K_inv_top = np.hstack((C, E))
    K_inv_bottom = np.hstack((E.T, F))
    K_inv = np.vstack((K_inv_top, K_inv_bottom))
    gh = np.vstack((g, h))
    sol = K_inv @ gh
    p_k = -sol[:n, :]
    # x_sol = x + p
    return p_k

def calc_alpha_k(A, W, b, p_k, x):
    # Calculate alpha_k by from (16.41)
    alpha_k = 1
    blocking_constraint_index = None
    for i in range(A.shape[0]):
        if i not in W and np.dot(A[i], p_k) < 0:
            temp_nom = b[i] - np.dot(A[i], x)
            temp_denom = np.dot(A[i], p_k)
            temp_alpha = temp_nom / temp_denom
            if temp_alpha < alpha_k:
                alpha_k = temp_alpha
                blocking_constraint_index = i
    return alpha_k, blocking_constraint_index

def problem_setup(M, y, starting_points):
    '''
        This function solves a QP with equality constraints using Schur-complement method

        Parameters:
            M:  m x n matrix
            y:  m-dim vector
            starting_points: list of n-dim vectors, initial starting points for QP-algorithm

        Returns:
            G_tilde:                  2n x 2n zero matrix, where G = M.T @ M fills the top left corner 
            c_tilde:                  2n x 1 vector, where c = -M.T @ y fills the top n entries
            starting_points_tilde:    list of 2n x 1 vectors, where x fills the top n entries and z (abs(x)) fills the bottom
            A:                        2n + 1 x 2n matrix, Constraint matrix where each row correspons to individual constraint
            b:                        2n + 1 x 1 vector, RHS of Constraint  
        '''
    
    m, n = M.shape
    
    G = np.dot(M.T, M)  # n x n 
    G_tilde = np.zeros((2*n, 2*n))
    G_tilde[:n, :n] = G

    c = -np.dot(M.T, y) # n x 1
    c_tilde = np.zeros((2*n, 1))
    c_tilde[:n] = c
    
    starting_points_tilde = []
    for x in starting_points:
        z = np.abs(x) # n x 1
        x_tilde = np.vstack([x, z])
        starting_points_tilde.append(x_tilde)
    
    constraint_1 = np.hstack([-np.eye(n), np.eye(n)])               # -x + z >= 0
    constraint_2 = np.hstack([np.eye(n), np.eye(n)])                # x + z >= 0 
    constraint_3 = np.hstack([np.zeros((1, n)), -np.ones((1, n))])  # -sum(z) >= -1

    A = np.vstack([constraint_1, constraint_2, constraint_3])
    b = np.hstack([np.zeros(2*n), [-1]]).reshape(-1, 1)
    
    return G_tilde, c_tilde, starting_points_tilde, A, b

def display_problem(G, c, starting_points, A, b):    
    print(f"Matrix G with shape {G.shape}:")
    print(print_array(G))
    
    print(f"\nVector c with shape {c.shape} (representation is transposed):")
    print(print_array(c.T))
    
    for i, starting_point in enumerate(starting_points):
        print(f"\nStarting point {i+1} with shape {starting_point.shape} (representation is transposed):")
        print(print_array(starting_point.T, decimals=1))
        
    print(f"\nMatrix A with shape {A.shape}:")
    print(print_array(A))
    
    print(f"\nVector b with shape {b.shape} (representation is transposed):")
    print(print_array(b.T))

In [4]:
def active_set_qp(G, c, A, b, starting_point, tol=1e-2, max_iter=100, print_iter=False):
    n = int(G.shape[0] / 2)
    m = int(n / 2)
    
    # Initialize variables
    x = starting_point
    W = set(get_initial_W(A, x, b))  # Working set of active constraints
    p_k = None
    # Add some random perturbation to G to avoid singularity
    noise = np.random.rand(*G.shape)
    G = G + noise
    
    for iteration in range(max_iter):
        if print_iter:
            if iteration != 0:
                print("\n")
            print(f"Current iteration: {iteration}")
            print(f"  x_{iteration} with shape {x.shape} (representation is transposed):")
            print(" " + print_array(x.T, decimals=4))
            print(f"  W_{iteration} = {W}")
        
        # Calculate p_k by solving the subproblem
        A_w = np.array([A[i] for i in W]) # Constraint matrix with only the constraints where i in W
        b_w = np.array([b[i] for i in W]) # RHS of constraint matrix with only the values b_i where i in W
        g_k = G @ x + c

        p_k = calc_pk(G, c, A_w, b_w, x)
        
        if print_iter:
            print(f"  p_{iteration} with shape {p_k.shape} (representation is transposed):")
            print(" " + print_array(p_k.T, decimals=4))
            
        # If p_k == 0, we need to calculate the multipliers
        if np.isclose(0, np.linalg.norm(p_k)): 
            lambda_w = np.linalg.lstsq(A_w.T, g_k, rcond=None)[0]
            if print_iter:
                print(f"  lambda_{iteration} with shape {lambda_w.shape} (representation is transposed):")
                print(" " + print_array(lambda_w.T, decimals=4))
                
            # Check if all Lagrange multipliers are non-negative
            if all(lambda_w >= -tol):
                print("Found optimal solution!")
                if not print_iter:
                    print(f"Iterations: {iteration}")
                print(f"x_opt = [" + print_array(x[:n].T) + "]")
                return x[:n]  # Optimal solution found
                
            # Identify the most negative Lagrange multiplier
            min_lambda_idx = np.argmin(lambda_w)
            min_lambda_val = list(W)[min_lambda_idx]
                
            # Update the working set by removing the corresponding constraint
            if print_iter:
                print(f"  Removing {min_lambda_val} from W!")
            W.remove(min_lambda_val)
    
        # if p_k != 0, we need to calculate alpha with a value in the interval [0, 1]
        else: 
            alpha_k, blocking_idx = calc_alpha_k(A, W, b, p_k, x)
            if print_iter:
                print(f"  alpha_{iteration} = {alpha_k}")
            
            # new x_k
            x = x + alpha_k * p_k.reshape(-1, 1)

            if blocking_idx:
                if print_iter:
                    print(f"  Adding blocking index {blocking_idx} to W!")
                W.add(blocking_idx)
        
    # end of max_iter
    print(f"Max iterations reached! Final x = [" + print_array(x[:n].T) + "]")
    return x[:n]

In [5]:
QP_problems = {}

for m in range(1, 6):
    I = (-1 ** m) * np.eye(m)
    O = np.zeros((m, m))
    M = np.hstack((I, O))

    y = np.random.randint(0, 6, size=(m, 1))
    
    starting_x = [np.zeros(2*m).reshape(-1, 1)]
    for i in range(2):
        # Generate random values for the starting point x
        x = np.random.rand(2*m, 1)

        # Normalize x to have an L1 norm less than 1
        x /= (np.sum(np.abs(x)) + 1e-6)  # Add a small epsilon to avoid division by zero
        starting_x.append(x)
    
    # Add the parameters and starting points (if any) to the dictionary
    QP_problems[f"Problem {m}"] = {
        "parameters": {
            "M": M,
            "y": y,
            "starting_points": starting_x
        }
    }
    
# Finally add the given example from the assignment
M_hat = np.array([
    [1, 1, 0, 0],
    [0, 0, 1, 1]
])
zero_matrix = np.zeros((2, 4))
M = np.zeros((10, 20))
y = np.array([1, -2, 3, -4, 5, -5, 4, -3, 2, -1])
y = y.reshape(-1, 1)
starting_x = []

for i in range(5):
    row_start = i * 2
    col_start = i * 4
    M[row_start:row_start+2, col_start:col_start+4] = M_hat
    
    # Generate random values for the starting point x
    x = np.random.rand(20, 1)

    # Normalize x to have an L1 norm less than 1
    x /= (np.sum(np.abs(x)) + 1e-6)  # Add a small epsilon to avoid division by zero
    starting_x.append(x)

QP_problems[f"Problem 6"] = {
        "parameters": {
            "M": M,
            "y": y,
            "starting_points": starting_x
    }
}

In [6]:
for problem_name, problem_data in QP_problems.items():
    print(f"Processing {problem_name}:")
    print("-"*21)
    # Access the parameters and reformulating the problem 
    paras = problem_data['parameters']
    G, c, starting_points, A, b = problem_setup(**paras)
    
    # Display the different variables of the QP
    display_problem(G, c, starting_points, A, b)
    
    # Solve the QP for the different starting points
    for i, starting_point in enumerate(starting_points):
        print("\n" + "-"*40)
        print(f"Trying to solve QP for starting point {i+1}:")
        print("-"*40)
        x_opt = active_set_qp(G, c, A, b, starting_point, print_iter=False)
    
    print("\n" + "="*135 + "\n")

Processing Problem 1:
---------------------
Matrix G with shape (4, 4):
 1 |  0 |  0 |  0
 0 |  0 |  0 |  0
 0 |  0 |  0 |  0
 0 |  0 |  0 |  0

Vector c with shape (4, 1) (representation is transposed):
 3 |  0 |  0 |  0

Starting point 1 with shape (4, 1) (representation is transposed):
 0.0 |  0.0 |  0.0 |  0.0

Starting point 2 with shape (4, 1) (representation is transposed):
 0.1 |  0.9 |  0.1 |  0.9

Starting point 3 with shape (4, 1) (representation is transposed):
 0.3 |  0.7 |  0.3 |  0.7

Matrix A with shape (5, 4):
-1 |  0 |  1 |  0
 0 | -1 |  0 |  1
 1 |  0 |  1 |  0
 0 |  1 |  0 |  1
 0 |  0 | -1 | -1

Vector b with shape (5, 1) (representation is transposed):
 0 |  0 |  0 |  0 | -1

----------------------------------------
Trying to solve QP for starting point 1:
----------------------------------------
Found optimal solution!
Iterations: 3
x_opt = [-1 | -0]

----------------------------------------
Trying to solve QP for starting point 2:
-------------------------------